In [ ]:
#!pip uninstall tensorflow
!pip install tensorflow

In [ ]:
!pip install scikeras
%pip install sktime==0.20 #0.33.1


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.0/17.0 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 47.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 48.8 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.0
    Uninstalling scikit-learn-1.6.0:
      Successfully uninstalled scikit-learn-1.6.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is th

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Input
from tensorflow.keras.models import Model
from scikeras.wrappers import KerasClassifier

from sklearn.model_selection import cross_val_score, StratifiedKFold

# Função para criar o modelo
def create_model():
    input_layer = Input(shape=(512, 1))  # Supondo 512 pontos de dados unidimensionais
    x = Conv1D(filters=64, kernel_size=3, activation='relu')(input_layer)
    x = MaxPooling1D(pool_size=2)(x)
    x = Flatten()(x)
    x = Dense(128, activation='relu')(x)
    predictions = Dense(6, activation='softmax')(x)  # Supondo 10 classes de saída

    model = Model(inputs=input_layer, outputs=predictions)
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

In [ ]:
model = KerasClassifier(build_fn=create_model, epochs=10, batch_size=32, verbose=0)
MAIN_DATA = "AllCandidasBloadOversample"
# MAIN_DATA = "AllCandidasBload"

In [ ]:
candida_data_train  = pd.read_csv(f"{MAIN_DATA}_TRAIN.csv", sep=",", header=None)
# 652 is the column of label
class_col = candida_data_train.shape[1]-1
print(f"class_col  {class_col}")
y_train = candida_data_train[class_col]
X_train = candida_data_train.drop(columns=[class_col])

candida_data_test  = pd.read_csv(f"{MAIN_DATA}_VAL.csv", sep=",",header=None)
y_val = candida_data_test[class_col]
X_val = candida_data_test.drop(columns=[class_col])

X_train_val = pd.concat([X_train, X_val])
y_train_val = pd.concat([y_train, y_val])

candida_data_test  = pd.read_csv(f"{MAIN_DATA}_TEST.csv", sep=",",header=None)
y_test = candida_data_test[class_col]
X_test = candida_data_test.drop(columns=[class_col])
# bacteria_data_val  = pd.read_csv(rootPath+valDataFrame, sep=",")
# y_val = bacteria_data_val['652']
# X_val = bacteria_data_val.drop(columns=['652'])

print(f"Treino {len(X_train_val)}")
print(f"Teste {len(X_test)}")

class_col  512
Treino 436
Teste 110


In [ ]:
y_test

,512
0,2.0
1,3.0
2,5.0
3,2.0
4,5.0
...,...
105,3.0
106,1.0
107,5.0
108,3.0


In [ ]:
print(np.unique(y_train_val))  # Deve mostrar valores entre 0 e 4
print(np.unique(y_test))    # Deve mostrar valores entre 0 e 4


[1. 2. 3. 4. 5. 6.]
[1. 2. 3. 4. 5. 6.]


In [ ]:
y_train_val = y_train_val - 1
y_test = y_test - 1

In [ ]:
print(np.unique(y_train_val))  # Deve mostrar valores entre 0 e 5
print(np.unique(y_test))    # Deve mostrar valores entre 0 e 5

[0. 1. 2. 3. 4. 5.]
[0. 1. 2. 3. 4. 5.]


In [ ]:
# from tensorflow.keras.utils import to_categorical

## Supondo que `train_labels` e `val_labels` sejam seus rótulos
# train_labels = to_categorical(y_train, num_classes=6)
# val_labels = to_categorical(y_test, num_classes=6)


In [ ]:
# Cross-validation
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
results = cross_val_score(model, X_train, y_train, cv=kfold)

/usr/local/lib/python3.11/dist-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/usr/local/lib/python3.11/dist-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/usr/local/lib/python3.11/dist-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/usr/local/lib/python3.11/dist-packages/scikeras/wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
/usr/local/lib/python3.11/dist-packages/scikeras

In [ ]:
results

array([0.48484848, 0.37878788, 0.36923077, 0.15384615, 0.16923077])

In [ ]:
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [ ]:
def create_models(models):
    models.append(('XGBClassifier',XGBClassifier(use_label_encoder=False, eval_metric='logloss')))
    svm = SVC(kernel='linear', random_state=42)
    models.append(('SVC',OneVsRestClassifier(svm)))
    models.append(('DecisionTreeClassifier',DecisionTreeClassifier(random_state=42)))
    models.append(('KNeighborsClassifier',KNeighborsClassifier(n_neighbors=5)))
    models.append(('RandomForestClassifier',RandomForestClassifier(n_estimators=100, random_state=42)))


    return models

In [ ]:
def create_dt_metrics():
    columns = ['XGBClassifier','SVC','DecisionTreeClassifier'
               ,'KNeighborsClassifier']
    df_precision = pd.DataFrame(data= None,
                           columns=columns)
    df_accuracy  = pd.DataFrame(data= None,
                           columns=columns)
    df_f1        = pd.DataFrame(data= None,
                           columns=columns)
    df_recall    = pd.DataFrame(data= None,
                           columns=columns)
    df_std       = pd.DataFrame(data= None,
                           columns=columns)
    return df_precision, df_accuracy, df_f1, df_recall, df_std

In [ ]:
import pandas as pd
from pickle import dump

def save_results(df_to_save, name, model, train_val_test):
    # Salva o modelo
    print("Saving model...")
    dump(model, open(f"{name}_{train_val_test}_model.pkl", 'wb'))

    # Calcula as médias e o desvio padrão
    print("Calculating mean and standard deviation metrics...")
    mean_metrics = df_to_save.mean(axis=0).to_frame(name='mean').T  # Calcula a média
    std_metrics = df_to_save.std(axis=0).to_frame(name='std').T  # Calcula o desvio padrão

    # Combina médias e desvios padrões em um único DataFrame
    summary_metrics = pd.concat([mean_metrics, std_metrics])

    # Salva os dados detalhados
    detailed_csv_file = f"df_{name}_{train_val_test}.csv"
    print(f"Saving detailed metrics to {detailed_csv_file}...")
    df_to_save.to_csv(detailed_csv_file, index=False)

    # Salva as médias e desvios padrões em um novo arquivo CSV
    summary_csv_file = f"df_{name}_{train_val_test}_avg_std.csv"
    print(f"Saving average and standard deviation metrics to {summary_csv_file}...")
    summary_metrics.to_csv(summary_csv_file, index=True)

    print("Metrics saved successfully!")
    print(f"Detailed metrics saved to: {detailed_csv_file}")
    print(f"Summary metrics (mean and std) saved to: {summary_csv_file}")


In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import RepeatedStratifiedKFold
import tensorflow as tf
import time

def save_results_old(df_to_save, name, model, train_val_test):
    from pickle import dump
    #save the model
    print("Saving model...")
    dump(model, open(f"{name}_{train_val_test}_model.pkl", 'wb'))
    print("Saving metrics...")
    print(f"Df accuracy: {df_to_save}")
    df_to_save.to_csv(f"df_{name}_{train_val_test}.csv", index=False)
    print("Accuracy of models:")
    print(df_to_save)

def calculate_metrics_per_class(y_val, y_pred, classes):
    cm = confusion_matrix(y_val, y_pred, labels=classes)
    metrics_per_class = []

    for i, class_label in enumerate(classes):
        TP = cm[i, i]
        FN = cm[i, :].sum() - TP
        FP = cm[:, i].sum() - TP
        TN = cm.sum() - (TP + FP + FN)

        accuracy = (TP + TN) / cm.sum() if cm.sum() > 0 else 0
        precision = TP / (TP + FP) if (TP + FP) > 0 else 0
        recall = TP / (TP + FN) if (TP + FN) > 0 else 0
        specificity = TN / (TN + FP) if (TN + FP) > 0 else 0
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

        metrics_per_class.append({
            'class': class_label,
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'f1_score': f1,
            'specificity': specificity
        })

    return metrics_per_class


# Função para calcular especificidade global
def calculate_global_specificity(y_val, y_pred, classes):
    cm = confusion_matrix(y_val, y_pred, labels=classes)
    specificity_per_class = []
    for i in range(len(classes)):
        TN = cm.sum() - (cm[i, :].sum() + cm[:, i].sum() - cm[i, i])  # True Negatives
        FP = cm[:, i].sum() - cm[i, i]  # False Positives
        specificity = TN / (TN + FP) if (TN + FP) > 0 else 0
        specificity_per_class.append(specificity)
    return np.nanmean(specificity_per_class)

In [ ]:
import joblib
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_multilabel_classification
import numpy as np

def cross_validate_and_save_best_model(model, X, y, cv=5, scoring='accuracy', filename='best_model.joblib'):
    # Configurar a validação cruzada
    skf = StratifiedKFold(n_splits=cv, shuffle=True, random_state=42)

    best_score = -np.inf
    best_model = None
    train_scores = []
    test_scores = []
    fit_times = []
    score_times = []

    for train_index, test_index in skf.split(X, y):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        # Treinar o modelo
        model.fit(X_train, y_train)

        # Avaliar o modelo
        train_score = model.score(X_train, y_train)
        test_score = model.score(X_test, y_test)

        train_scores.append(train_score)
        test_scores.append(test_score)

        # Salvar o melhor modelo
        if test_score > best_score:
            best_score = test_score
            best_model = model

    # Salvar o melhor modelo
    joblib.dump(best_model, filename)

    # Carregar o modelo salvo
    loaded_model = joblib.load(filename)

    # Retornar as métricas e o modelo treinado
    cv_results = {
        'train_score': train_scores,
        'test_score': test_scores,
    }
    return cv_results, loaded_model




In [ ]:
y_train_val

,512
0,5.0
1,0.0
2,3.0
3,1.0
4,1.0
...,...
104,2.0
105,5.0
106,1.0
107,5.0


In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import RepeatedStratifiedKFold, RepeatedKFold
import tensorflow as tf
import time

# Configuração do RepeatedStratifiedKFold
kf = RepeatedKFold(n_splits=5, n_repeats=2, random_state=42)


# Função para calcular métricas por classe
def calculate_metrics_per_class(y_val, y_pred, classes):
    cm = confusion_matrix(y_val, y_pred, labels=classes)
    metrics_per_class = []

    for i, class_label in enumerate(classes):
        TP = cm[i, i]
        FN = cm[i, :].sum() - TP
        FP = cm[:, i].sum() - TP
        TN = cm.sum() - (TP + FP + FN)

        accuracy = (TP + TN) / cm.sum() if cm.sum() > 0 else 0
        precision = TP / (TP + FP) if (TP + FP) > 0 else 0
        recall = TP / (TP + FN) if (TP + FN) > 0 else 0
        specificity = TN / (TN + FP) if (TN + FP) > 0 else 0
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

        metrics_per_class.append({
            'class': class_label,
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'f1_score': f1,
            'specificity': specificity
        })

    return metrics_per_class


# Função para calcular especificidade global
def calculate_global_specificity(y_val, y_pred, classes):
    cm = confusion_matrix(y_val, y_pred, labels=classes)
    specificity_per_class = []
    for i in range(len(classes)):
        TN = cm.sum() - (cm[i, :].sum() + cm[:, i].sum() - cm[i, i])  # True Negatives
        FP = cm[:, i].sum() - cm[i, i]  # False Positives
        specificity = TN / (TN + FP) if (TN + FP) > 0 else 0
        specificity_per_class.append(specificity)
    return np.nanmean(specificity_per_class)


# Função para salvar resultados em um CSV
def save_to_csv(data, filename):
    try:
        df = pd.DataFrame(data)
        df.to_csv(filename, index=False)
        print(f"Resultados salvos em {filename}")
    except Exception as e:
        print(f"Erro ao salvar o arquivo: {e}")


# Função para treinar e avaliar os modelos
def train_and_evaluate(models, X_train_val, Y_train_val, kf, output_global=f"results_global_{MAIN_DATA}.csv", output_class=f"results_per_class_{MAIN_DATA}.csv"):
    all_global_metrics = []
    all_class_metrics = []

    with tf.device('/device:GPU:0'):
        for name, model in models:
            print(f"Treinando modelo: {name}")
            fold_idx = 1
            start_time = time.time()
            best_score = -np.inf
            for train_idx, val_idx in kf.split(X_train_val, Y_train_val):
                # Divisão de treino e validação
                if isinstance(X_train_val, pd.DataFrame):
                    X_train, X_val = X_train_val.iloc[train_idx], X_train_val.iloc[val_idx]
                else:
                    X_train, X_val = X_train_val[train_idx], X_train_val[val_idx]

                if isinstance(Y_train_val, pd.Series):
                    y_train, y_val = Y_train_val.iloc[train_idx], Y_train_val.iloc[val_idx]
                else:
                    y_train, y_val = Y_train_val[train_idx], Y_train_val[val_idx]

                # Treinamento do modelo
                print(X_train)
                model.fit(X_train, y_train)

                # Predição
                # y_pred = model.predict(X_val).argmax(axis=1)
                y_pred = model.predict(X_val)
                print(f" y_pred: {y_pred}")

                # Classes únicas
                classes = sorted(set(Y_train_val))

                # Métricas por classe
                class_metrics = calculate_metrics_per_class(y_val, y_pred, classes)
                for metrics in class_metrics:
                    metrics.update({'model': name, 'fold': fold_idx})
                    all_class_metrics.append(metrics)

                # Métricas globais
                global_metrics = {
                    'model': name,
                    'fold': fold_idx,
                    'accuracy': accuracy_score(y_val, y_pred),
                    'precision': precision_score(y_val, y_pred, average='macro'),
                    'recall': recall_score(y_val, y_pred, average='macro'),
                    'f1_score': f1_score(y_val, y_pred, average='macro'),
                    'specificity': calculate_global_specificity(y_val, y_pred, classes)
                }
                all_global_metrics.append(global_metrics)
                        # Salvar o melhor modelo
                if global_metrics["accuracy"] > best_score:
                    best_score = global_metrics["accuracy"]
                    best_model = model

                # Salvar o melhor modelo
                joblib.dump(best_model, f"best_model_{name}")
                fold_idx += 1

            end_time = time.time()
            all_global_metrics.append({'time_seconds':end_time - start_time})
            print(f"Tempo total para {name}: {end_time - start_time:.2f} segundos")

    # Salvar resultados
    save_to_csv(all_global_metrics, output_global)
    save_to_csv(all_class_metrics, output_class)

    return all_global_metrics, all_class_metrics


In [ ]:
if __name__=="__main__":
    models =  []
    models = create_models(models)
    if tf.config.list_physical_devices('GPU'):
        print("Using MirroredStrategy")
        strategy = tf.distribute.MirroredStrategy()
    else:  # Use the Default Strategy
        print("Using Default Strategy")
        strategy = tf.distribute.get_strategy()
    with strategy.scope():
        # Do something interesting
        print(tf.Variable(1.))
        train_and_evaluate(models, X_train_val, y_train_val, kf)

Using MirroredStrategy
MirroredVariable:{
  0: <tf.Variable 'Variable:0' shape=() dtype=float32, numpy=1.0>
}
Treinando modelo: XGBClassifier
        0       1       2      3       4       5       6      7       8    \
1    1535.0  1045.0  1384.0  594.4  1536.0  1044.0  1384.0  594.4  1535.0   
2    1525.0  1036.0  1295.0  603.6  1528.0  1036.0  1298.0  603.4  1525.0   
3    1595.0  1041.0  1566.0  640.3  1595.0  1041.0  1569.0  640.1  1596.0   
4    1574.0  1030.0  1478.0  628.4  1574.0  1030.0  1482.0  628.8  1575.0   
5    1503.0  1004.0  1417.0  605.1  1503.0  1004.0  1417.0  605.3  1502.0   
..      ...     ...     ...    ...     ...     ...     ...    ...     ...   
102  1549.0  1059.0  1316.0  597.0  1547.0  1060.0  1317.0  597.6  1547.0   
103  1573.0  1072.0  1357.0  617.5  1569.0  1069.0  1356.0  617.5  1573.0   
106  1574.0  1036.0  1673.0  623.2  1575.0  1036.0  1675.0  623.0  1574.0   
107  1350.0   866.3  1539.0  502.7  1350.0   867.1  1540.0  502.7  1351.0   
108  1549.0

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [15:24:17] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


 y_pred: [5 5 1 0 5 4 2 4 3 1 0 1 3 3 2 1 1 2 5 4 1 2 3 1 3 5 0 2 2 3 0 2 0 5 5 4 4
 1 5 1 3 2 4 3 5 3 2 0 3 5 1 2 1 5 4 3 4 0 3 5 2 3 1 4 2 4 5 3 5 5 3 2 2 0
 3 4 2 3 0 0 3 2 2 0 4 2 2 5]
        0       1       2      3       4       5       6      7       8    \
0    1312.0   867.3   998.6  536.4  1313.0   867.5   998.8  536.6  1313.0   
1    1535.0  1045.0  1384.0  594.4  1536.0  1044.0  1384.0  594.4  1535.0   
2    1525.0  1036.0  1295.0  603.6  1528.0  1036.0  1298.0  603.4  1525.0   
4    1574.0  1030.0  1478.0  628.4  1574.0  1030.0  1482.0  628.8  1575.0   
6    1448.0   952.1  1340.0  570.0  1448.0   951.7  1340.0  570.2  1448.0   
..      ...     ...     ...    ...     ...     ...     ...    ...     ...   
104  1632.0  1073.0  1904.0  679.8  1636.0  1074.0  1930.0  681.6  1635.0   
105  1365.0   894.9  1333.0  534.4  1366.0   894.3  1334.0  534.4  1365.0   
106  1574.0  1036.0  1673.0  623.2  1575.0  1036.0  1675.0  623.0  1574.0   
107  1350.0   866.3  1539.0  502.7  1350.

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [15:24:19] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


 y_pred: [1 1 5 4 4 5 2 5 4 2 3 1 4 2 1 3 1 5 4 2 3 0 4 0 3 3 4 1 0 5 3 0 0 5 3 4 5
 4 1 4 2 0 3 3 4 0 3 5 1 5 1 2 4 5 4 3 5 1 1 4 3 4 2 3 3 0 3 5 0 2 2 3 3 4
 1 2 1 2 1 2 3 5 1 3 4 1 2]
        0       1       2      3       4       5       6      7       8    \
0    1312.0   867.3   998.6  536.4  1313.0   867.5   998.8  536.6  1313.0   
1    1535.0  1045.0  1384.0  594.4  1536.0  1044.0  1384.0  594.4  1535.0   
3    1595.0  1041.0  1566.0  640.3  1595.0  1041.0  1569.0  640.1  1596.0   
4    1574.0  1030.0  1478.0  628.4  1574.0  1030.0  1482.0  628.8  1575.0   
5    1503.0  1004.0  1417.0  605.1  1503.0  1004.0  1417.0  605.3  1502.0   
..      ...     ...     ...    ...     ...     ...     ...    ...     ...   
104  1632.0  1073.0  1904.0  679.8  1636.0  1074.0  1930.0  681.6  1635.0   
105  1365.0   894.9  1333.0  534.4  1366.0   894.3  1334.0  534.4  1365.0   
106  1574.0  1036.0  1673.0  623.2  1575.0  1036.0  1675.0  623.0  1574.0   
107  1350.0   866.3  1539.0  502.7  1350.0 

...

 y_pred: [5 3 5 1 5 5 2 5 2 1 3 3 1 3 1 3 3 0 2 4 2 4 1 0 1 5 3 5 1 0 3 1 2 2 2 4 0
 5 0 3 4 2 1 4 1 2 0 4 5 0 5 1 3 5 1 3 4 5 3 3 4 3 5 5 3 1 5 1 0 2 5 0 3 4
 1 1 5 3 5 1 2 3 5 0 0 5 1]
        0       1       2      3       4       5       6      7       8    \
0    1312.0   867.3   998.6  536.4  1313.0   867.5   998.8  536.6  1313.0   
1    1535.0  1045.0  1384.0  594.4  1536.0  1044.0  1384.0  594.4  1535.0   
2    1525.0  1036.0  1295.0  603.6  1528.0  1036.0  1298.0  603.4  1525.0   
3    1595.0  1041.0  1566.0  640.3  1595.0  1041.0  1569.0  640.1  1596.0   
5    1503.0  1004.0  1417.0  605.1  1503.0  1004.0  1417.0  605.3  1502.0   
..      ...     ...     ...    ...     ...     ...     ...    ...     ...   
102  1549.0  1059.0  1316.0  597.0  1547.0  1060.0  1317.0  597.6  1547.0   
104  1632.0  1073.0  1904.0  679.8  1636.0  1074.0  1930.0  681.6  1635.0   
105  1365.0   894.9  1333.0  534.4  1366.0   894.3  1334.0  534.4  1365.0   
106  1574.0  1036.0  1673.0  623.2  1575.0 

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [15:24:37] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


 y_pred: [1 0 3 2 4 2 3 4 4 5 0 0 2 3 0 4 1 0 1 5 2 0 5 4 3 4 1 0 3 2 4 0 0 5 3 4 3
 4 5 0 3 4 1 5 0 1 4 0 2 5 3 3 4 2 1 4 4 3 4 2 4 1 0 0 5 3 3 1 3 0 2 2 2 2
 3 2 2 1 2 0 0 4 4 2 1 2 3]
        0       1       2      3       4       5       6      7       8    \
0    1312.0   867.3   998.6  536.4  1313.0   867.5   998.8  536.6  1313.0   
1    1535.0  1045.0  1384.0  594.4  1536.0  1044.0  1384.0  594.4  1535.0   
2    1525.0  1036.0  1295.0  603.6  1528.0  1036.0  1298.0  603.4  1525.0   
4    1574.0  1030.0  1478.0  628.4  1574.0  1030.0  1482.0  628.8  1575.0   
5    1503.0  1004.0  1417.0  605.1  1503.0  1004.0  1417.0  605.3  1502.0   
..      ...     ...     ...    ...     ...     ...     ...    ...     ...   
102  1549.0  1059.0  1316.0  597.0  1547.0  1060.0  1317.0  597.6  1547.0   
103  1573.0  1072.0  1357.0  617.5  1569.0  1069.0  1356.0  617.5  1573.0   
105  1365.0   894.9  1333.0  534.4  1366.0   894.3  1334.0  534.4  1365.0   
106  1574.0  1036.0  1673.0  623.2  1575.0 

/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [15:24:40] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


 y_pred: [1 5 3 1 4 5 3 0 1 4 3 2 1 2 1 5 3 3 2 3 1 4 5 3 0 0 2 5 3 1 5 1 4 2 2 4 1
 5 1 1 3 0 5 1 4 1 3 3 4 1 5 2 4 2 5 5 3 3 0 1 5 5 3 5 0 2 0 0 2 3 0 2 0 5
 1 3 0 3 5 5 2 2 0 4 5 2 5]
Tempo total para XGBClassifier: 27.67 segundos
Treinando modelo: SVC
        0       1       2      3       4       5       6      7       8    \
1    1535.0  1045.0  1384.0  594.4  1536.0  1044.0  1384.0  594.4  1535.0   
2    1525.0  1036.0  1295.0  603.6  1528.0  1036.0  1298.0  603.4  1525.0   
3    1595.0  1041.0  1566.0  640.3  1595.0  1041.0  1569.0  640.1  1596.0   
4    1574.0  1030.0  1478.0  628.4  1574.0  1030.0  1482.0  628.8  1575.0   
5    1503.0  1004.0  1417.0  605.1  1503.0  1004.0  1417.0  605.3  1502.0   
..      ...     ...     ...    ...     ...     ...     ...    ...     ...   
102  1549.0  1059.0  1316.0  597.0  1547.0  1060.0  1317.0  597.6  1547.0   
103  1573.0  1072.0  1357.0  617.5  1569.0  1069.0  1356.0  617.5  1573.0   
106  1574.0  1036.0  1673.0  623.2  1575.0  1036.0 

In [ ]:
# gpu(kf, models)

In [ ]:
def save_final_result(df, model_name, phase, suffix=""):
    """
    Salva os resultados do teste em um arquivo CSV.
    """
    filename = f"{model_name}_{phase}_results{suffix}.csv"
    df.to_csv(filename, index=False)
    print(f"Resultados salvos em {filename}")

def test_final_model(X_test, y_test, models):
    """
    Testa modelos finais nos dados de teste e salva as métricas em arquivos CSV.
    """
    for name, model in models:
        print(f"\nTestando modelo: {name}")

        # DataFrames para métricas globais e por classe
        df_test = pd.DataFrame(columns=['accuracy', 'recall', 'precision', 'f1_score', 'specificity', 'execution_time'])
        df_test_per_class = pd.DataFrame(columns=['class', 'accuracy', 'precision', 'recall', 'f1_score', 'specificity'])

        # Início da medição de tempo
        start_time = time.time()
        y_pred = model.predict(X_test)
        end_time = time.time()
        execution_time = end_time - start_time

        # Cálculo das métricas globais
        accuracy = accuracy_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred, average='macro')
        precision = precision_score(y_test, y_pred, average='macro')
        f1 = f1_score(y_test, y_pred, average='macro')

        # Especificidade global
        classes = sorted(set(y_test))
        specificity = calculate_metrics_per_class(y_test, y_pred, classes)
        avg_specificity = sum([m['specificity'] for m in specificity]) / len(specificity)

        # Adiciona métricas globais ao DataFrame
        df_test = pd.DataFrame([{
            'accuracy': accuracy,
            'recall': recall,
            'precision': precision,
            'f1_score': f1,
            'specificity': avg_specificity,
            'execution_time': execution_time
        }])

        # Adiciona métricas por classe ao DataFrame
        df_test_per_class = pd.DataFrame(specificity)

        # Exibe os resultados
        print("\nMétricas Globais:")
        print(df_test)
        print("\nMétricas por Classe:")
        print(df_test_per_class)

        # Salva os resultados
        save_final_result(df_test, name, f'test_{MAIN_DATA}')
        save_final_result(df_test_per_class, name, 'test', f'_per_class_{MAIN_DATA}')
        print("Teste concluído!")

In [ ]:
test_final_model(X_test, y_test, models)


Testando modelo: XGBClassifier

Métricas Globais:
   accuracy    recall  precision  f1_score  specificity  execution_time
0  0.963636  0.963938   0.962418  0.962397     0.992811        0.386708

Métricas por Classe:
   class  accuracy  precision    recall  f1_score  specificity
0    0.0  1.000000   1.000000  1.000000  1.000000     1.000000
1    1.0  0.972727   0.944444  0.894737  0.918919     0.989011
2    2.0  1.000000   1.000000  1.000000  1.000000     1.000000
3    3.0  0.981818   0.888889  1.000000  0.941176     0.978723
4    4.0  0.972727   0.941176  0.888889  0.914286     0.989130
5    5.0  1.000000   1.000000  1.000000  1.000000     1.000000
Resultados salvos em XGBClassifier_test_AllCandidasBloadOversample_results.csv
Resultados salvos em XGBClassifier_test_results_per_class_AllCandidasBloadOversample.csv
Teste concluído!

Testando modelo: SVC

Métricas Globais:
   accuracy    recall  precision  f1_score  specificity  execution_time
0  0.981818  0.983292   0.981969  0.982164  